In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from Utils import Utils
from Constants import Constants
from Propensity_socre_network import Propensity_socre_network
from Utils import Utils
from PS_Manager import PS_Manager
from PS_Treated_Generator import PS_Treated_Generator

from GAN import Generator, Discriminator
from sklearn.neighbors import NearestNeighbors

from GAN_Manager import GAN_Manager
from Utils import Utils

from matplotlib import pyplot
from torch.autograd.variable import Variable
from collections import OrderedDict
from scipy.special import expit


In [8]:
ori_data = np.loadtxt("Dataset/Twin_data.csv", delimiter=",",skiprows=1)

  # Define features
x = ori_data[:,:30]
no, dim = x.shape
    
  # Define potential outcomes
potential_y = ori_data[:, 30:]
  # Die within 1 year = 1, otherwise = 0
potential_y = np.array(potential_y < 9999,dtype=float)    
print(potential_y[:5,:])

  



[[0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]]


In [9]:
## Assign treatment
coef = np.random.uniform(-0.01, 0.01, size = [dim,1])
prob_temp = expit(np.matmul(x, coef) + np.random.normal(0,0.01, size = [no,1]))
print(prob_temp)


[[0.52850723]
 [0.52334743]
 [0.50391847]
 ...
 [0.51152641]
 [0.50501045]
 [0.52185088]]


In [15]:
prob_t = prob_temp/(2*np.mean(prob_temp))
prob_t[prob_t>1] = 1
    
t = np.random.binomial(1,prob_t,[no,1])
t = t.reshape([no,])

print(t[:10])


[1 0 0 1 0 0 0 1 1 1]


In [18]:
## Define observable outcomes
y = np.zeros([no,1])
y = np.transpose(t) * potential_y[:,1] + np.transpose(1-t) * potential_y[:,0]    
y = np.reshape(np.transpose(y), [no, ])

print(y.shape)
print(y[:10])

(11400,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
  ## Train/test division
train_rate = 0.8
idx = np.random.permutation(no)
train_idx = idx[:int(train_rate * no)]
test_idx = idx[int(train_rate * no):]
    
train_x = x[train_idx,:]
train_t = t[train_idx]
train_y = y[train_idx]
train_potential_y = potential_y[train_idx,:]
print(train_potential_y[:5,:])

print(train_potential_y.shape)

test_x = x[test_idx,:]
test_potential_y = potential_y[test_idx,:]
        
#return train_x, train_t, train_y, train_potential_y, test_x, test_potential_y

[[0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]]
(9120, 2)


In [23]:
test_t = t[test_idx]

In [25]:
print(test_t.shape)
print(test_x.shape)

(2280,)
(2280, 30)


In [26]:
print(train_t.shape)
print(train_x.shape)

(9120,)
(9120, 30)


In [31]:
train_t_id = (train_t == 1)
t_1 = train_t[train_t_id]
print(t_1.shape)
print(train_t[~train_t_id].shape)

(4464,)
(4656,)
